# Módulo Deep Learning
## Actividad 2: Reinforcement Learning: **Frozen lake problem**

- David Moreno Vituri
- Ricardo Sánchez Olivares
- Íñigo Alegre Jiménez

# Actividad Reinforcemente Learning

Resolver el problema del Frozen lake de OpenAI Gym. Documentación: https://www.gymlibrary.dev/environments/toy_text/frozen_lake/

## Objetivos
- Conseguir movermos aleatoriamente hasta cumplir el objetivo
- Conseguir que el agente aprenda con Q-learning
- (Opcional) Probar con otros hiperparámetros
- (Opcional) Modificar la recompensa

## Consideraciones
- No hay penalizaciones
- Si el agente cae en un "hole", entonces done = True y se queda atascado sin poder salir (al igual que ocurre cuando llega al "goal")

## Normas a seguir

- Se debe entregar un **ÚNICO GOOGLE COLAB notebook** (archivo .ipynb) que incluya las instrucciones presentes y su **EJECUCIÓN!!!**.
- Poner el nombre del grupo en el nombre del archivo y el nombre de todos los integrantes del grupo al inicio del notebook.

## Criterio de evaluación

- Seguimiento de las normas establecidas en la actividad.
- Corrección en el uso de algoritmos, modelos y formas idiomáticas en Python.
- El código debe poder ejecutarse sin modificación alguna en Google Colaboratory.

In [48]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Instalamos librerías**

In [2]:
!pip install numpy==1.24.4
!pip install gym==0.17.3

In [3]:
import gym
import numpy as np
from time import sleep
from IPython.display import clear_output
import random as rd

##**Definición del entorno**

In [4]:
# Definimos el entorno
env = gym.make('FrozenLake-v0', desc=None, map_name="4x4", is_slippery=False)

In [5]:
# Fijamos una semilla
seed_value = 42
env.seed(seed_value)
np.random.seed(seed_value)

In [6]:
env.reset() # En este caso, empieza desde la misma posición inicial
print(env.render())


SFFF
FHFH
FFFH
HFFG
None


In [7]:
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space Discrete(4)
State Space Discrete(16)


Acciones posibles:
* 0: izquierda
* 1: abajo
* 2: derecha
* 3: arriba

In [8]:
# Identificador de estado
state = env.s
print("State:", state)

State: 0


## **¡Nos movemos aleatoriamente!**

In [38]:
steps = 0
env.reset()
env.render()


SFFF
FHFH
FFFH
HFFG


In [39]:
# Acciones: 0=izquierda, 1=abajo, 2=derecha, 3=arriba
action = 2
state, reward, done, info = env.step(action)

print("State:", state)
print(state, reward, done, info)

env.s = state
env.render()

steps += 1

print(f"Step: {steps}")

# hemos comprobado que efectivamente, cuando caemos en un hueco, no podemos movernos más

State: 1
1 0.0 False {'prob': 1.0}
  (Right)
SFFF
FHFH
FFFH
HFFG
Step: 1


Recompensas:
* Cada paso: 0
* Caerse por el acantilado: 0 pero pone done = True
* Llegar al destino: +1 y pone done = True

## Nos movemos aleatoriamente hasta llegar al objetivo

In [40]:
done = False  # inicializamos done a False indicando que no se ha terminado

env.reset()

timestep, caidas, final = 0, 0, 0

while not ((done) & (final==1)):

  action = env.action_space.sample() # con "sample" elegimos una de las acciones del action_space
  state, final, done, info = env.step(action) # con "step" realizamos la acción elegida

  if ((done == True)&(final==0)):
      caidas += 1 # sumamos una penalización si el muñeco se cae a un hueco
      env.reset() # ponemos el muñeco en el punto de partida porque no se puede mover dentro de un hueco

  timestep += 1

print("Timesteps taken: {}".format(timestep))
print("Penalties incurred: {}".format(caidas))
env.render() # visualizamos el estado final

Timesteps taken: 932
Penalties incurred: 128
  (Right)
SFFF
FHFH
FFFH
HFFG


In [43]:
# vamos a mostrar como consigue llegar hasta la meta de manera aleatoria
class bcolors:
    RED= '\u001b[31m'
    GREEN= '\u001b[32m'
    RESET= '\u001b[0m'

done = False

env.reset()

timestep, caidas, final = 0, 0, 0
total_reward = 0

while not ((done) & (final==1)):
  action = env.action_space.sample() # con "sample" elegimos una de las acciones del action_space
  state, final, done, info = env.step(action) # con "step" realizamos la acción elegida

  if ((done == True)&(final==0)):
      caidas += 1 # sumamos una penalización si el muñeco se cae a un hueco
      env.reset() # ponemos el muñeco en el punto de partida porque no se puede mover dentro de un hueco

  total_reward += final
  timestep += 1

  # Print each step
  clear_output(wait=True)
  env.render()
  print("\nIteración: {}".format(timestep))
  print("Caidas: {}".format(caidas))
  if final == 0:
    print(f"Recompensa actual: {bcolors.RED}{final}{bcolors.RESET}")
  else:
    print(f"Recompensa actual: {bcolors.GREEN}{final}{bcolors.RESET}")
  print("")
  print('Estado actual', state)
  sleep(.5)

print("Pasos dados: {}".format(timestep))

  (Right)
SFFF
FHFH
FFFH
HFFG

Iteración: 1826
\Caidas: 253
Recompensa actual: 1.0

Estado actual 15
Pasos dados: 1826
Caidas: 253


## Q-Learning entrenamiento

Comenzamos definiendo la función epsilon greedy policy, la cual nos permite indicar al modelo cuanto debe explorar y cuanto debe 'explotar' la opción más rentable. Cuanto mayor sea epsilon, más explora el modelo y viceversa.

In [45]:
#trade-off entre explorar y explotar
def epsilon_greedy_policy(epsilon, q_table, state, env):
  if rd.random() < epsilon:
      action = env.action_space.sample() #explorar
  else:
      action = np.argmax(q_table[state]) #explotar
  return action

Ahora pasamos a entrenar y actualizar la Q-table para que más tarde el modelo sepa el camino más efectivo y rápido para alcanzar el objetivo.

In [46]:
# Hyperparameters
alpha = 0.2 # tasa de aprendizaje
gamma = 0.7 # tasa de descuento
epsilon = 0.15 # greedy policy

# For plotting metrics
all_timestep = []
all_penalties = []

episodes = 100000

q_table = np.zeros([env.observation_space.n, env.action_space.n])

for i in range(episodes):
  state = env.reset()

  timestep, caidas, final = 0, 0, 0
  done = False

  while not ((done) & (final==1)):

    # Elegimos y ejecutamos la acción
    action = epsilon_greedy_policy(epsilon, q_table, state, env) # aplicamos la greedy policy
    next_state, final, done, info = env.step(action) # tomamos la acción elegida

    # actualizamos la Q-table
    old_value = q_table[state, action] # en la Q-table, tomamos el valor Q de la acción elegida para el estado actual
    next_max = np.max(q_table[next_state]) # en la Q-table, tomamos el máximo entre los valores Q para el nuevo estado
    new_value = (1 - alpha) * old_value + alpha * (final + gamma * next_max) # actualizamos el valor Q
    q_table[state, action] = new_value

    if ((done == True)&(final==0)):
        caidas += 1 # sumamos una penalización si el muñeco se cae a un hueco
        env.reset() # ponemos el muñeco en el punto de partida porque no se puede mover dentro de un hueco

    state = next_state

  if i % 100 == 0:
    clear_output(wait=True)
    print(f"Episodio: {i+1}")

  timestep += 1

clear_output(wait=True)
print(f"Episodio: {i+1}")
print("¡Entrenamiento finalizado!")

Episodio: 100000
¡Entrenamiento finalizado!


Una vez hemos actualizado la Q-table ya no necesitamos explorar (el camino más eficaz se encuentra ahí). Es por ello que cambiamos la greedy policy de manera que la acción elegida sea siempre escoger como siguiente movimiento aquel que tiene mayor Q-value.

In [47]:
class bcolors:
  RED= '\u001b[31m'
  GREEN= '\u001b[32m'
  RESET= '\u001b[0m'

env.reset()
env.render()
done = False

timestep, caidas, final = 0, 0, 0
total_reward = 0

while not ((done) & (final==1)):
  action = np.argmax(q_table[state])
  state, final, done, info = env.step(action) # con "step" realizamos la acción elegida

  if ((done == True)&(final==0)):
    caidas += 1 # sumamos una penalización si el muñeco se cae a un hueco
    env.reset() # ponemos el muñeco en el punto de partida porque no se puede mover dentro de un hueco

  total_reward += final
  timestep += 1

  # Print each step
  sleep(3)
  clear_output(wait=True)
  env.render()

  if final == 0:
    print(f"Recompensa actual: {bcolors.RED}{final}{bcolors.RESET}")
  else:
    print(f"Recompensa actual: {bcolors.GREEN}{final}{bcolors.RESET}")
  print("")
  print('Estado actual', state)
  sleep(.5)

print("Timesteps taken: {}".format(timestep))
print("Total de caidas: {}".format(caidas))

  (Right)
SFFF
FHFH
FFFH
HFFG
Recompensa actual: 1.0

Estado actual 15
Timesteps taken: 7
Total de caidas: 0
